# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
from timm.scheduler import CosineLRScheduler
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# effnet1 = timm.create_model('efficientnet_b1', pretrained=True)
# # M7:block[5], M8:block[6], M9:conv_head
# effnet1

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'

TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/pred'
SCORE_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/score'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)
os.makedirs(SCORE_DIR, exist_ok=True)

# copy config
# shutil.copy('./config.yaml', OUTPUT_ROOT)

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    
    # train_source
    #dev_train_paths = dev_train_source_paths
    
    # train_tgt
    train_paths = []
    train_tgt_paths = []
    # 0~2 section
    for section in range(0,3):
        # sourceを収集
        tmp_train_paths = [path for path in dev_train_source_paths if f'section_0{section}_source' in path] # (1000,)
        train_paths += tmp_train_paths
        # targetを収集
        per_section_tgt = [path for path in dev_train_target_paths if f'section_0{section}_target' in path] # (3,)
        # sourceの分だけ増やす
        train_tgt_paths += [per_section_tgt]*len(tmp_train_paths) # (1000, 3)  
    
    # add_dev_train
    add_dev_train_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_train_paths = sorted(add_dev_train_paths)
    print(f'adddev_train_size : {len(add_dev_train_paths)}', end=', ')
    add_dev_train_source_paths = [path for path in add_dev_train_paths if 'source' in path]
    print(f'source_size : {len(add_dev_train_source_paths)}', end=', ')
    add_dev_train_target_paths = [path for path in add_dev_train_paths if 'target' in path]
    print(f'target_size : {len(add_dev_train_target_paths)}')

    # add_train_source
    #add_train_paths = add_dev_train_source_paths
    
    # train_tgt
    add_train_paths = []
    add_train_tgt_paths = []
    # 3~5 section
    for section in range(3,6):
        # sourceを収集
        tmp_add_train_paths = [path for path in add_dev_train_source_paths if f'section_0{section}_source' in path] # (1000,)
        add_train_paths += tmp_add_train_paths
        # targetを収集
        per_section_tgt = [path for path in add_dev_train_target_paths if f'section_0{section}_target' in path] # (3,)
        # sourceの分だけ増やす
        add_train_tgt_paths += [per_section_tgt]*len(tmp_add_train_paths) # (1000, 3)
        
        
    # sourceのpathを参照して，対応するtgt_sectionのサンプルpathsを作成
    
    
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # train
    train_source = train_paths + add_train_paths
    train_target = train_tgt_paths + add_train_tgt_paths
    train_paths = [{'source': train_source[i], 'target': train_target[i]} for i in range(len(train_source))]
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = train_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3114, source_size : 3105, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [10]:
machine_types

['pump', 'gearbox', 'slider', 'ToyCar', 'valve', 'ToyTrain', 'fan']

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    model = Model(n_out=36, n_classes=6)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    scheduler = CosineLRScheduler(optimizer, t_initial=100, lr_min=1e-4, 
                                  warmup_t=10, warmup_lr_init=5e-5, warmup_prefix=True)
    output_dict, model, pred_df, score_df = modeler.run_training(model, dataloaders_dict, writer, optimizer, scheduler)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    score_out_path = f'{SCORE_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dict, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    pred_df.to_csv(pred_out_path)
    score_df.to_csv(score_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

In [12]:
#run(machine_types[0], dev_paths)

In [13]:
for machine_type in machine_types[0:]:
    run(machine_type, dev_paths)

2021-11-08 12:26:39,609 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-11-08 12:26:39,610 - 00_train.py - INFO - MAKE DATA_LOADER
2021-11-08 12:26:39,615 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 600/600 [01:01<00:00,  9.79it/s]
2021-11-08 12:29:53,835 - pytorch_modeler.py - INFO - epoch:1/100, tr_loss:3.398730, src_loss:3.335540, src_mean_auc:0.588900, tgt_loss:3.367212, tgt_mean_auc:0.539933, mean_auc:0.564417,


,AUC,pAUC
Source_0,0.628800,0.603158
Source_1,0.533200,0.523158
Source_2,0.604700,0.532632
Target_0,0.556000,0.508421
Target_1,0.460500,0.485263
Target_2,0.603300,0.507368
mean,0.564417,0.526667
h_mean,0.558286,0.524226


100%|██████████| 24/24 [00:54<00:00,  2.26s/it]
2021-11-08 12:30:48,581 - pytorch_modeler.py - INFO - epoch:2/100, tr_loss:2.828372, src_loss:3.335540, src_mean_auc:0.588900, tgt_loss:3.367212, tgt_mean_auc:0.539933, mean_auc:0.564417,


...

,AUC,pAUC
Source_0,0.646900,0.593158
Source_1,0.895200,0.694737
Source_2,0.663000,0.544737
Target_0,0.542000,0.544737
Target_1,0.680700,0.536842
Target_2,0.565800,0.522632
mean,0.665600,0.572807
h_mean,0.648459,0.567487


100%|██████████| 24/24 [00:54<00:00,  2.26s/it]
2021-11-08 12:55:19,667 - pytorch_modeler.py - INFO - epoch:24/100, tr_loss:0.462264, src_loss:0.881663, src_mean_auc:0.735033, tgt_loss:0.919165, tgt_mean_auc:0.596167, mean_auc:0.665600,


,AUC,pAUC
Source_0,0.646900,0.593158
Source_1,0.895200,0.694737
Source_2,0.663000,0.544737
Target_0,0.542000,0.544737
Target_1,0.680700,0.536842
Target_2,0.565800,0.522632
mean,0.665600,0.572807
h_mean,0.648459,0.567487


  0%|          | 0/24 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
#run(machine_types[0], dev_paths)

# run